## Exploratory Data Analysis of mmDoppler Data

In [1]:
import sys
sys.path.append("../")

import polars as pl
import plotly.express as px

In [2]:
micro_df = pl.read_parquet("../data/micro_df.parquet")
micro_df_subset = pl.read_parquet("../data/micro_df_subset.parquet")

In [ ]:
# Identify the Stationary / Clutter bin
all_doppler = micro_df.select(pl.col("dopplerIdx").list.explode()).to_series()
print(f"Min Doppler: {all_doppler.min()}")
print(f"Max Doppler: {all_doppler.max()}")

px.histogram(all_doppler.to_numpy(), title="Doppler Distribution").show() 

Min Doppler: -64
Max Doppler: 63


#### Check for class imbalance
Activity distribution across all frames

In [ ]:
from helper.analysis_utils import ClassDistributionAnalyzer

dist_analyzer = ClassDistributionAnalyzer(micro_df)

# Compute and Display Distribution
dist_df = dist_analyzer.compute_distribution()
print(dist_df)

# Plot Micro activites (Typing, Eating, etc.) -> IDs 10-18
dist_analyzer.plot_distribution(dist_df).show()


shape: (9, 4)
┌──────────┬───────┬────────────────┬────────────┐
│ activity ┆ count ┆ Activity Name  ┆ Percentage │
│ ---      ┆ ---   ┆ ---            ┆ ---        │
│ i64      ┆ u32   ┆ str            ┆ f64        │
╞══════════╪═══════╪════════════════╪════════════╡
│ 10       ┆ 4635  ┆ Phone Typing   ┆ 18.352075  │
│ 12       ┆ 4546  ┆ Sitting        ┆ 17.999683  │
│ 11       ┆ 3600  ┆ Laptop Typing  ┆ 14.254039  │
│ 18       ┆ 2353  ┆ Drinking Water ┆ 9.316598   │
│ 15       ┆ 2287  ┆ Playing Guitar ┆ 9.055274   │
│ 13       ┆ 2153  ┆ Eating         ┆ 8.524707   │
│ 16       ┆ 2116  ┆ Brushing Teeth ┆ 8.378207   │
│ 17       ┆ 2099  ┆ Combing Hair   ┆ 8.310896   │
│ 14       ┆ 1467  ┆ Phone Talking  ┆ 5.808521   │
└──────────┴───────┴────────────────┴────────────┘


#### Doppler Analysis: Range-Doppler Heatmap visualization

In [23]:
from helper.analysis_utils import visualize_activity_grid_inline

# Check if dist_analyzer is defined
if 'dist_analyzer' not in locals():
    from helper.analysis_utils import ClassDistributionAnalyzer
    dist_analyzer = ClassDistributionAnalyzer(micro_df)

fig_grid = visualize_activity_grid_inline(micro_df, dist_analyzer, n_frames=10)
print(fig_grid.show())


None


#### Duration analysis
Calculate the average duration of each activity class. Real-time systems usually need a buffer (e.g., "detect typing for 3 seconds before alerting"). Knowing the typical duration helps us set this buffer size.

- [Note] Micro-Doppler activites were collected at 2 FPS

##### Calculate average duration of Drinking Water Activity

In [22]:
drink_duration_df = micro_df.filter(pl.col("activity") == 18)

In [ ]:
# Check if the data is sorted by datetime
print(f"Is sorted: {drink_duration_df["datetime"].is_sorted()}")

# Sort by datetime if not sorted
if not drink_duration_df["datetime"].is_sorted():
    drink_duration_df = drink_duration_df.sort("datetime")


Is sorted: True


In [24]:
# Show "jumps" for out-of-order data
fig = px.scatter(
    x=range(len(drink_duration_df)), 
    y=drink_duration_df['datetime'].to_list(),
    title="Data Timeline (Index vs Time)",
    labels={'x': 'Row Index', 'y': 'Timestamp'}
)
print(fig.show())

#[Conclusion] It shows the data was collected only for <2 days. Lacks variability. But atleast it is not out of order.

None
